In [ ]:
def main_fn(path_config_file, extra_args={}):
    # Read the configration file
    weight_noise, bias_noise, mSampleLog, mParamsNN_list, (type_baseline, data_set_file, out_file) = load_config_file(
        path_config_file, extra_args)
    reducedSampleLog = SampleLog(xTrain=None, xTrainExtra=None, uTrain=None, xnextTrain=None,
                                 lowU_train=mSampleLog.lowU_train, highU_train=mSampleLog.highU_train,
                                 xTest=None, xTestExtra=None, uTest=None, xnextTest=None,
                                 lowU_test=mSampleLog.lowU_test, highU_test=mSampleLog.highU_test,
                                 env_name=mSampleLog.env_name, env_extra_args=mSampleLog.env_extra_args,
                                 m_rng=mSampleLog.m_rng, seed_number=mSampleLog.seed_number, qp_indx=mSampleLog.qp_indx,
                                 qp_base=mSampleLog.qp_base, n_state=mSampleLog.n_state, n_control=mSampleLog.n_control,
                                 actual_dt=mSampleLog.actual_dt, disable_substep=mSampleLog.disable_substep,
                                 control_policy=mSampleLog.control_policy, n_rollout=mSampleLog.n_rollout)

mSampleLog is literally just the train data file
reducedSampleLog is like mSampleLog, but with less unecessary information

In [ ]:
    # Print the sample log info
    print("###### SampleLog ######")
    print(reducedSampleLog)

    # Training data
    xTrainList = np.asarray(mSampleLog.xTrain)
    (xTrainExtra, coloc_set) = mSampleLog.xTrainExtra
    xnextTrainList = np.asarray(mSampleLog.xnextTrain)
    (_, num_traj_data, trajectory_length) = mSampleLog.disable_substep
    coloc_set = jnp.array(coloc_set)

    # Testing data
    xTest = jnp.asarray(mSampleLog.xTest)
    xTestNext = jnp.asarray(mSampleLog.xnextTest)

    # DIctionary for logging training details
    final_log_dict = {i: {} for i in range(len(num_traj_data))}
    final_params_dict = {i: {} for i in range(len(num_traj_data))}

    for i, n_train in tqdm(enumerate(num_traj_data),  total=len(num_traj_data)):
        # The total number of point in this trajectory
        total_traj_size = n_train*trajectory_length

        # Dictionary to save the loss and the optimal params per radom seed
        dict_loss_per_seed = {mParamsNN.seed_init: {}
                              for mParamsNN in mParamsNN_list}
        dict_params_per_seed = {
            mParamsNN.seed_init: None for mParamsNN in mParamsNN_list}

        for _, mParamsNN in tqdm(enumerate(mParamsNN_list), total=len(mParamsNN_list), leave=False):

            # Some logging execution
            dbg_msg = "###### Hyper parameters: {} ######\n".format(
                type_baseline)
            dbg_msg += "{}\n".format(mParamsNN)
            tqdm.write(dbg_msg)

            # Build the neural network, the update, loss function and paramters of the neural network structure
            rng_gen, opt, (params, m_pen_eq_k, m_pen_ineq_k, m_lagr_eq_k, m_lagr_ineq_k), pred_xnext,\
                loss_fun, update, update_lagrange, loss_fun_constr, train_with_constraints = build_learner(
                    mParamsNN, type_baseline)

            # Initialize the optimizer with the initial parameters
            opt_state = opt.init(params)

            # Get the frequency at which to evaluate on the testing
            high_freq_record_rg = int(
                mParamsNN.freq_accuracy[0]*mParamsNN.num_gradient_iterations)
            high_freq_val = mParamsNN.freq_accuracy[1]
            low_freq_val = mParamsNN.freq_accuracy[2]
            update_freq = jnp.array([(j % high_freq_val) == 0 if j <= high_freq_record_rg else ((j % low_freq_val) == 0 if j < mParamsNN.num_gradient_iterations-1 else True)
                                     for j in range(mParamsNN.num_gradient_iterations)])

            # Dictionary to save the loss, the rollout error, and the colocation accuracy
            dict_loss = {'total_loss_train': list(), 'rollout_err_train': list(), 'total_loss_test': list(
            ), 'rollout_err_test': list(), 'coloc_err_train': list(), 'coloc_err_test': list()}

            # Store the best parameters attained by the learning process so far
            best_params = None 				# Weights returning best loss over the training set
            best_test_loss = None 			# Best test loss over the training set
            best_train_loss = None 			# Best training loss
            best_constr_val = None 			# Constraint attained by the best params
            best_test_noconstr = None 		# Best mean squared error without any constraints
            # Iteration since the last best weights values (iteration in terms of the loss evaluation period)
            iter_since_best_param = None
            number_lagrange_update = 0 		# Current number of lagrangian and penalty terms updates
            number_inner_iteration = 0 		# Current number of iteration the latest update
            # Period over which no improvement yields the best solution
            patience = mParamsNN.patience
            tqdm.write('\t\tEarly stopping criteria = {}'.format(patience > 0))

            # Iterate to learn the weight of the neural network
            for step in tqdm(range(mParamsNN.num_gradient_iterations), leave=False):

                # Random key generator for batch data
                rng_gen, subkey = jax.random.split(rng_gen)
                idx_train = jax.random.randint(subkey, shape=(
                    mParamsNN.batch_size,), minval=0, maxval=total_traj_size)

                # Sample and put into the GPU memory the batch-size data
                xTrain, xTrainNext = jnp.asarray(xTrainList[idx_train, :]), jnp.asarray(
                    xnextTrainList[:, idx_train, :])

                # Sample batches of the colocation data sets
                rng_gen, subkey = jax.random.split(rng_gen)
                idx_coloc_train = jax.random.randint(subkey, shape=(
                    mParamsNN.batch_size,), minval=0, maxval=coloc_set.shape[0])
                batch_coloc = coloc_set[idx_coloc_train]

                # Update the parameters of the NN and the state of the optimizer
                params, opt_state = update(params, opt_state, xTrainNext, xTrain, pen_eq_k=m_pen_eq_k, pen_ineq_sq_k=m_pen_ineq_k,
                                           lagr_eq_k=m_lagr_eq_k if m_lagr_eq_k.shape[
                                               0] <= 0 else m_lagr_eq_k[idx_coloc_train],
                                           lagr_ineq_k=m_lagr_ineq_k if m_lagr_ineq_k.shape[
                                               0] <= 0 else m_lagr_ineq_k[idx_coloc_train],
                                           extra_args_colocation=(batch_coloc, None, None))

                # If it is time to evaluate the models do it
                if update_freq[number_inner_iteration]:
                    if not train_with_constraints:  # In case there is no constraints -> Try to also log the constraints incurred by the current neural structure
                        loss_tr, (spec_data_tr, coloc_ctr) = loss_fun_constr(
                            params, xTrainNext, xTrain, gradient_regularization=gradient_regularization)
                        loss_te, (spec_data_te, coloc_cte) = loss_fun_constr(
                            params, xTestNext, xTest, extra_args_colocation=(coloc_set, None, None), gradient_regularization=gradient_regularization)
                    else:
                        loss_tr, (spec_data_tr, coloc_ctr) = loss_fun(
                            params, xTrainNext, xTrain)
                        loss_te, (spec_data_te, coloc_cte) = loss_fun(params, xTestNext, xTest, pen_eq_k=m_pen_eq_k, pen_ineq_sq_k=m_pen_ineq_k,
                                                                      lagr_eq_k=m_lagr_eq_k, lagr_ineq_k=m_lagr_ineq_k,
                                                                      extra_args_colocation=(coloc_set, None, None))

                    # Log the value obtained by evaluating the current model
                    dict_loss['total_loss_train'].append(float(loss_tr))
                    dict_loss['total_loss_test'].append(float(loss_te))
                    dict_loss['rollout_err_train'].append(spec_data_tr)
                    dict_loss['rollout_err_test'].append(spec_data_te)
                    dict_loss['coloc_err_train'].append(coloc_ctr)
                    dict_loss['coloc_err_test'].append(coloc_cte)

                    # Initialize the parameters for the best model so far
                    if number_inner_iteration == 0:
                        best_params = params
                        best_test_noconstr = jnp.mean(
                            spec_data_te[:, 1])  # Rollout mean
                        best_test_loss = loss_te
                        best_train_loss = loss_tr
                        best_constr_val = coloc_cte
                        iter_since_best_param = 0

                    # Check if the validation metrics has improved
                    if loss_te < best_test_loss:
                        best_params = params
                        best_test_loss = loss_te
                        best_test_noconstr = jnp.mean(spec_data_te[:, 1])
                        best_constr_val = coloc_cte
                        best_train_loss = loss_tr if loss_tr < best_train_loss else best_train_loss
                        iter_since_best_param = 0
                    else:
                        best_train_loss = loss_tr if loss_tr < best_train_loss else best_train_loss
                        iter_since_best_param += 1

                # Period at which we save the models
                if number_inner_iteration % mParamsNN.freq_save == 0:

                    # Update for the current seed what is the lost and the best params found so far
                    dict_loss_per_seed[mParamsNN.seed_init] = dict_loss
                    final_log_dict[i] = dict_loss_per_seed

                    # Update the best params found so far
                    dict_params_per_seed[mParamsNN.seed_init] = best_params
                    final_params_dict[i] = dict_params_per_seed

                    # Create the log file
                    mLog = LearningLog(env_name=mSampleLog.env_name, env_extra_args=mSampleLog.env_extra_args, baseline=type_baseline,
                                       sampleLog=reducedSampleLog, nn_hyperparams=mParamsNN_list, loss_evol=final_log_dict, learned_weights=final_params_dict,
                                       data_set_file=data_set_file)

                    # Save the current information in the file and close the file
                    mFile = open(out_file+'.pkl', 'wb')
                    pickle.dump(mLog, mFile)
                    mFile.close()

                    # Debug message to bring on the consolde
                    dbg_msg = '[Iter[{}][{}], N={}]\t train : {:.6f} | Loss test : {:.6f}\n'.format(
                        step, number_lagrange_update, n_train, loss_tr, loss_te)
                    dbg_msg += '\t\tBest Loss Function : Train = {:.6f} | Test = {:.6f}, {:.6f} | Constr = {}\n'.format(
                        best_train_loss, best_test_loss, best_test_noconstr, best_constr_val.round(6))
                    dbg_msg += '\t\tPer rollout loss train : {} | Loss test : {}\n'.format(
                        spec_data_tr[:, 1].round(6), spec_data_te[:, 1].round(6))
                    dbg_msg += '\t\tPer rollout EQ Constraint Train : {} | Test : {}\n'.format(
                        spec_data_tr[:, 2].round(6), spec_data_te[:, 2].round(6))
                    dbg_msg += '\t\tPer rollout INEQ Constraint Train : {} | Test : {}\n'.format(
                        spec_data_tr[:, 3].round(6), spec_data_te[:, 3].round(6))
                    dbg_msg += '\t\tColocotion loss : {}\n'.format(
                        coloc_cte.round(8))
                    tqdm.write(dbg_msg)

                # Update the number of iteration
                number_inner_iteration += 1

                # If the patience periode has been violated, try to break
                if patience > 0 and iter_since_best_param > patience:

                    # Debug message
                    tqdm.write('####################### EARLY STOPPING [{}][{}] #######################'.format(
                        step, number_lagrange_update))
                    tqdm.write('Best Loss Function : Train = {:.6f} | Test = {:.6f}, {:.6f}'.format(
                        best_train_loss, best_test_loss, best_test_noconstr))

                    # If there is no constraints then break
                    if not train_with_constraints:
                        break

                    # Check if the constraints threshold has been attained
                    if best_constr_val[1] < mParamsNN.pen_constr['tol_constraint_eq'] and best_constr_val[2] < mParamsNN.pen_constr['tol_constraint_ineq']:
                        tqdm.write('Constraints satisfied: [eq =  {}], [ineq = {}]\n'.format(
                            best_constr_val[1], best_constr_val[2]))
                        tqdm.write(
                            '##############################################################################\n')
                        break

                    # If the constraints threshold hasn't been violated, update the lagrangian and penalty coefficients
                    m_pen_eq_k, m_pen_ineq_k, m_lagr_eq_k, m_lagr_ineq_k = \
                        update_lagrange(params, pen_eq_k=m_pen_eq_k, pen_ineq_sq_k=m_pen_ineq_k,
                                        lagr_eq_k=m_lagr_eq_k, lagr_ineq_k=m_lagr_ineq_k, extra_args_colocation=(coloc_set, None, None))

                    # Update the new params to be the best params
                    params = best_params

                    # UPdate the number of inner iteration since the last lagrangian terms update
                    number_inner_iteration = 0

                    # Update the number of lagrange update so far
                    number_lagrange_update += 1

                    # Reinitialize the optimizer
                    opt_state = opt.init(params)

                    # Some printing for debig
                    tqdm.write('Update Penalty : [eq = {:.4f}, lag_eq = {:.4f}] | [ineq = {:.4f}, lag_ineq = {:.4f}]'.format(
                        m_pen_eq_k, jnp.sum(jnp.abs(m_lagr_eq_k)), m_pen_ineq_k, jnp.sum(m_lagr_ineq_k)))

            # Once the algorithm has converged, collect and save the data and params
            dict_loss_per_seed[mParamsNN.seed_init] = dict_loss
            final_log_dict[i] = dict_loss_per_seed
            # Here we update the weights with the noise
            key = jax.random.PRNGKey(0)
            # print(best_params) # TODO: Remove this
            best_params_noisy = {}
            for it_key, it_value in best_params.items():
                best_params_noisy[it_key] = {}
                for w_or_b, w_or_b_value in it_value.items():
                    if w_or_b == 'w':
                        best_params_noisy[it_key][w_or_b] = add_gaussian_noise(key, w_or_b_value, scale=weight_noise)
                    elif w_or_b == 'b':
                        best_params_noisy[it_key][w_or_b] = add_gaussian_noise(key, w_or_b_value, scale=bias_noise)
            dict_params_per_seed[mParamsNN.seed_init] = best_params_noisy
            final_params_dict[i] = dict_params_per_seed

            # Cretae the log
            mLog = LearningLog(env_name=mSampleLog.env_name, env_extra_args=mSampleLog.env_extra_args, baseline=type_baseline,
                               sampleLog=reducedSampleLog, nn_hyperparams=mParamsNN_list, loss_evol=final_log_dict, learned_weights=final_params_dict,
                               data_set_file=data_set_file)

            # Save the current information
            mFile = open(out_file+'.pkl', 'wb')
            pickle.dump(mLog, mFile)
            mFile.close()
